### Import Required Libraries and Set Up Environment Variables

In [95]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [96]:
# Set environment variables from the .env in the local environment
load_dotenv()
NYT_API_KEY = os.getenv("NYT_API_KEY")
TMDB_API_KEY = os.getenv("TMDB_API_KEY")

In [97]:
type(NYT_API_KEY)

str

### Access the New York Times API

In [98]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name: "Movies" AND type_of_material: "Review" AND headline: "love"'

# Use a sort filter, sort by newest
sort = "newest"
# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# query_url = (f"{url}api-key={NYT_API_KEY}&q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}")
query_url = (
    f"{url}api-key={NYT_API_KEY}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [99]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    # page_query = f"&page={page}"
    query_url = f"{query_url}&page={str(page)}"
    # Make a "GET" request and retrieve the JSON
    reviews_list = requests.get(query_url).json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        reviews = response.json()
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page + 1}")
    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"No results found for page {page + 1}: {e}")
        break
        

No results found for page 1: 'response'


In [100]:

# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [104]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list = reviews
df_reviews = pd.json_normalize(reviews_list)
df_reviews

,page,results,total_pages,total_results
0,1,"[{'adult': False, 'backdrop_path': '/q4GVfF2Z4...",31,617


In [103]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include "review" to avoid cutting title early
def extract_title(headline):
    start_index = headline.find('\u2018') + len('\u2018')
    end_index = headline.find('review') + len('review')
    if end_index != -1:
        end_index += len('review')
    else:
        return headline  # Return the original headline if " Review" is not found
    return headline[start_index:end_index]
# Create the new "title" column and apply the extract_title function
df_reviews['title'] = df_reviews["headline.main"].apply(extract_title)
df_reviews[['headline.main', 'title']].head()

KeyError: 'headline.main'

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df_reviews['title'].to_list()
title_list

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + TMDB_API_KEY

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)

   # Include a try clause to search for the full movie details. 
    # Use the except clause to print out a statement if a movie is not found.
    try:
        # Get movie id
        movie_id = response.json()["results"][0]["id"]

        # Make a request for the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}"
        movie_details_response = requests.get(movie_details_url)

        # Execute "GET" request with url
        movie_details = movie_details_response.json()

        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_details["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_details["title"],
            "overview": movie_details["overview"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })

        # Print out the title that was found
        print(f"Found details for movie: {title}")

    except IndexError:
        print(f"Movie not found: {title}")

# Print the tmdb_movies_list
for movie in tmdb_movies_list:
    print(movie)



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame table
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Print the DataFrame
df_tmdb_movies

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, on="title", how="inner")
# mergedDF = pd.merge(df_reviews, df_tmdb_movies, on="title")
# Print the merged DataFrame
merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
# Loop through characters to remove
    for char in columns_to_fix:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
merged_df


In [ ]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)


In [ ]:
# Delete duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Print the DataFrame after deleting duplicates and resetting index
merged_df


In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)